In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

sequence_lengths = [128, 256, 512]
packet_loss_prob = 0.1
ts = 0.1
T = 10
masking_sequence = np.array([1, -1, 1, -1])

def generate_sequence(n):
    return np.random.choice([-1, 1], size=n)

def simulate_packet_loss(sequence, p):
    n = len(sequence)
    n_loss = int(p * n)
    indices = np.random.choice(n, n_loss, replace=False)
    sequence[indices] = 0
    return sequence

def apply_masking(sequence, mask):
    m = len(mask)
    for j in range(len(sequence) // m):
        for i in range(m):
            idx = j * m + i
            if idx < len(sequence):
                sequence[idx] *= mask[i]
    return sequence

def calculate_metrics(sequence, T, ts):
    n = len(sequence)
    throughput = np.sum(np.abs(sequence)) / T
    latency = n * ts
    signal_power = np.count_nonzero(sequence)
    noise_power = np.count_nonzero(sequence == 0)
    sinr = signal_power / (noise_power + 1e-9)
    packet_loss_rate = noise_power / n
    return throughput, latency, sinr, packet_loss_rate

results = []

for n in sequence_lengths:
    seq = generate_sequence(n)
    seq = simulate_packet_loss(seq, packet_loss_prob)
    seq = apply_masking(seq, masking_sequence.copy())
    
    throughput, latency, sinr, plr = calculate_metrics(seq, T, ts)
    
    results.append({
        "Length": n,
        "Throughput (bps)": round(throughput, 2),
        "Latency (s)": round(latency, 2),
        "Signal Quality (SINR)": round(sinr, 2),
        "Packet Loss Rate": round(plr, 2)
    })
    
    plt.figure()
    plt.plot(seq)
    plt.title(f"WCDMA Sequence Length = {n}")
    plt.xlabel("Index")
    plt.ylabel("Value")
    plt.grid(True)
    plt.show()

df = pd.DataFrame(results)
print(df)

metrics = ["Throughput (bps)", "Latency (s)", "Signal Quality (SINR)", "Packet Loss Rate"]
for metric in metrics:
    plt.figure()
    plt.bar(df["Length"].astype(str), df[metric])
    plt.title(metric)
    plt.xlabel("Sequence Length")
    plt.ylabel(metric)
    plt.grid(True)
    plt.show()